# DecMeg2014 - Decoding the Human Brain

### Documentación

Repositorio de Kaggle: 
 https://www.kaggle.com/c/decoding-the-human-brain

Repositorio de Github (Ejemplos de códigos en Matlab y Python):
https://github.com/FBK-NILab/DecMeg2014

### Importación de librerías

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scipy.io import loadmat
import seaborn as sns

### Coger los datos de los archivos .mat

Creamos una función "create_features" que se encargará de devolver los valores X. En esta función, se realiza todo el procesamiento de los datos. Los preprocesamiento que realiza son:

    1. Paso de la lista X de 3D a 2D. Poniendo como filas el primer elemento (las diferentes pruebas) y como columnas se hace un join del segundo elemento(series temporales o canales) y del tercer elemento(valores de cada serie temporal)
    
    2. Cogemos una ventana de valores
    
    3. Normalizamos los valores restandolos por la media y dividiéndolo por la desviación típica

In [2]:
def create_features(XX, tmin, tmax, sfreq, tmin_original=-0.5):
    """Creation of the feature space:
    - restricting the time window of MEG data to [tmin, tmax]sec.
    - Concatenating the 306 timeseries of each trial in one long
      vector.
    - Normalizing each feature independently (z-scoring).
    """
    print("Applying the desired time window.")
    beginning = np.round((tmin - tmin_original) * sfreq).astype(np.int)
    end = np.round((tmax - tmin_original) * sfreq).astype(np.int)
    XX = XX[:, :, beginning:end].copy()

    print("2D Reshaping: concatenating all 306 timeseries.")
    XX = XX.reshape(XX.shape[0], XX.shape[1] * XX.shape[2])

    print("Features Normalization.")
    XX -= XX.mean(0)
    XX = np.nan_to_num(XX / XX.std(0))

    return XX

Cogemos los datos de los ficheros train_subject13.mat, del 13 al 16. En esta etapa se realizan varios pasos:

    1. Creamos una lista X_train y y_train que contendrán todos los datos para realizar el entrenamiento. 
    
    2. Por cada uno de los ficheros, llamamos a la función create_features para que realice el correspondiente preprocesamiento de los datos.
    
    3. Una vez tenemos los datos del fichero, los agregamos a la lista.
    
    4. Una vez tenemos todas las listas en la lista, creamos un array realizando un join de las listas dentro de la lista. Para eso utilizamos la función vstack de la librería numpy

In [3]:
    subjects_train = range(13, 14) # use range(13, 16) for all subjects
print("Training on subjects", subjects_train )

# We throw away all the MEG data outside the first 0.5sec from when
# the visual stimulus start:
tmin = 0.0
tmax = 0.500
print("Restricting MEG data to the interval [%s, %s]sec." % (tmin, tmax))

X_train = []
y_train = []
X_test = []
ids_test = []

print("Creating the trainset.")
for subject in subjects_train:
    filename = './data/data/train_subject%02d.mat' % subject
    print("Loading", filename)
    data = loadmat(filename, squeeze_me=True)
    XX = data['X']
    yy = data['y']
    sfreq = data['sfreq']
    tmin_original = data['tmin']
    print("Dataset summary:")
    print("XX:", XX.shape)
    print("yy:", yy.shape)
    print("sfreq:", sfreq)

    XX = create_features(XX, tmin, tmax, sfreq)

    X_train.append(XX)
    y_train.append(yy)

X_train = np.vstack(X_train)
y_train = np.concatenate(y_train)
print("Trainset:", X_train.shape)

Training on subjects range(13, 14)
Restricting MEG data to the interval [0.0, 0.5]sec.
Creating the trainset.
Loading ./data/data/train_subject13.mat
Dataset summary:
XX: (588, 306, 375)
yy: (588,)
sfreq: 250.0
Applying the desired time window.
2D Reshaping: concatenating all 306 timeseries.
Features Normalization.
Trainset: (588, 38250)


### Visualización de los datos

In [11]:
print(X_train.shape)
first_probe = X_train[0,:]
print(first_probe)
senors_values = []
for i in range(1,306):
    

(588, 38250)
[-1.8112224  -1.0874811  -0.18253571 ... -0.02355056  0.02517387
  0.09659942]


In [12]:
print(dataframe)
#sns.distplot(dataframe.iloc[0:100,0:375],kde=True,hist=True)

        0         1         2         3         4         5         6      \
0   -1.811222 -1.087481 -0.182536 -0.466642 -1.105666 -0.972519 -2.179797   
1   -0.680744 -0.542822 -0.578040  0.530787  0.271257 -0.706857 -1.821322   
2   -0.519984 -0.911802  0.202178  0.126008 -0.554339 -0.910988 -1.346741   
3   -0.711059  0.986628  0.665046 -0.811154 -0.626254 -0.379954 -0.749373   
4   -0.949496 -0.373295 -1.284967 -0.534028  0.959178  0.264727  2.180007   
5   -0.181002 -0.862479  0.570032  1.072724  0.489955 -0.082830  1.174180   
6    2.492688  1.151223  0.563009  1.893177  0.973079  0.798032 -0.575617   
7    0.854221  0.419142  1.067379  0.542577  0.884601  1.502928  1.355720   
8   -0.738382 -0.908667  0.036003  0.001355 -0.937850 -0.439793  0.979969   
9   -0.058762 -0.738514  0.342903  0.388052 -0.352417 -0.584514  0.084709   
10  -0.729575 -1.194956  0.374196 -0.141888  0.939628  1.310000 -0.323790   
11   2.020984  2.459413  1.487146  0.341238  0.632361  1.410035  0.889798   